In [50]:
import polars as pl
import json

from pathlib import Path

In [80]:
cmap_root = Path("C:/Users/fred/Data/foundata/cmap")

hh = (
    pl.read_csv(cmap_root / "household.csv", ignore_errors=True)
    .fill_nan(-9)
    .fill_null(-9)
)
person = (
    pl.read_csv(cmap_root / "person.csv", ignore_errors=True).fill_nan(-9).fill_null(-9)
)
trips = (
    pl.read_csv(cmap_root / "gps_place.csv", ignore_errors=True)
    .fill_nan(-9)
    .fill_null(-9)
)
vehicles = (
    pl.read_csv(cmap_root / "vehicle.csv", ignore_errors=True)
    .fill_nan(-9)
    .fill_null(-9)
)

In [81]:
hh.tail()

sampno,travdate,travday,hhsize,hhveh,transprob,transprob_o,hlive,restylast,restylast_o,restylast_zip,hhinc2,homeown,homeown_o,futuresurvey,incendonate,resty,resty_o,hhinc,download,source,source_o,hhtrips,phase,wthhfin
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64
70100991,"""2018-10-04""",5,3,3,1,-1,2,2,-1,60827,-1,3,-1,1,0,1,-1,2,1,2,-1,9,0,647.0892
70100992,"""2018-09-25""",3,3,3,1,-1,34,-1,-1,-1,-1,2,-1,1,0,2,-1,8,2,97,-9,4,0,305.085
70100993,"""2018-09-10""",2,2,3,3,-1,25,-1,-1,-1,-1,1,-1,1,-7,1,-1,7,2,7,-1,8,0,615.7184
70100998,"""2019-04-23""",3,1,0,97,-9,2,3,-1,60624,-1,3,-1,2,0,3,-1,2,2,97,-9,6,0,74.5194
70100999,"""2019-04-24""",4,1,1,3,-1,35,-1,-1,-1,-1,3,-1,1,0,1,-1,1,2,97,-9,5,0,313.6208


In [ ]:
hh = hh.with_columns(
    pl.col("travdate").str.split("-").list.get(0).str.to_integer().alias("year"),
    pl.col("travdate").str.split("-").list.get(1).str.to_integer().alias("month"),
)

with open("cmap/hh_dictionary.json") as f:
    hh_mapper = json.load(f)
hh_mapper = {c: {int(k): v for k, v in d.items()} for c, d in hh_mapper.items()}

# for col, mapping in hh_mapper.items():
# print(col)
# assert col in hh.columns
# assert set(mapping.keys()) == set(hh.select(pl.col(col)).unique())

hh.with_columns(
    pl.col(col).replace(mapping, return_dtype=pl.String)
    for col, mapping in hh_mapper.items()
)

C:\Users\fred\AppData\Local\Temp\ipykernel_14216\3861108546.py:15: DeprecationWarning: the `return_dtype` parameter for `replace` is deprecated. Use `replace_strict` instead to set a return data type while replacing values.
(Deprecated in version 1.0.0)
  hh.with_columns(pl.col(col).replace(mapping, return_dtype=pl.String) for col, mapping in hh_mapper.items())


InvalidOperationError: conversion from `str` to `i64` failed in column 'literal' for 1 out of 1 values: ["Appropriate skip"]

In [76]:
hh.select(pl.col("source")).contains(1)

AttributeError: 'DataFrame' object has no attribute 'contains'

In [ ]:
# import json

# maps = pl.read_csv(cmap_root / "data_dictionary.csv")

# for table, name in zip(
#     ["HOUSEHOLD", "PERSON", "GPS_DAY", "VEHICLE"],
#     ["hh", "person", "trip", "veh"]
# ):
#     data = maps.filter(pl.col("TABLE") == table)
#     mapper = {}
#     for i, frame in data.group_by("NAME"):
#         column = str(i[0]).lower()
#         mapper[column] = {}
#         keys = frame.select(pl.col("VALUE")).rows()
#         values = frame.select(pl.col("LABEL")).rows()
#         for k, v in zip(keys, values):
#             mapper[column][int(k[0])] = str(v[0])

#     with open(f"cmap/{name}_dictionary.json", "w", encoding="utf-8") as f:
#         json.dump(mapper, f, ensure_ascii=False, indent=4)